In [1]:
import dask
import dask.array as da
import numpy as np
import operator

### inputs

In [2]:
m = 1000
mc = 100

In [3]:
A = da.random.normal(0, 1, (m,m), chunks=mc)
b = da.random.normal(0, 1./m, m, chunks=mc)
x0 = None
M = None
name = None

### before iteration

In [4]:
def block_cg_initialize(A, b, M, x0, name=None):
    token = name or dask.base.tokenize(A)
    itertoken = 'cg-iter-0-' + token

    b = da.from_array(b, chunks=b.chunks, name='b-' + itertoken)

    if x0 is None:
        r = 1 * b
        x = 0 * b
    else:
        r = b - A.dot(x0)
        x = x0

    Mr = r if M is None else M.dot(r)

    r = da.from_array(r, chunks=b.chunks, name='r-' + itertoken)
    x = da.from_array(x, chunks=b.chunks, name='x-' + itertoken)
    p = da.from_array(Mr, chunks=b.chunks, name='p-' + itertoken)
    resnrm2 = r.dot(Mr)
    nm_resnrm2 = 'resnrm2-' + itertoken
    dsk_resnrm2 = {nm_resnrm2: resnrm2.name,}

    dsk = dask.sharedict.merge(A.dask, x.dask, r.dask, Mr.dask, p.dask, resnrm2.dask)
#     dsk.update_with_key(dsk_resnrm2, nm_resnrm2)
    x, r, p, resnrm2 = dask.persist(x, r, p, resnrm2)
    
    return dsk, x, resnrm2

In [5]:
def init2(A, b, M, x0, name=None):
    token = name or dask.base.tokenize(A)
    itertoken = 'cg-iter-0-' + token
    nblks_1d = len(b.chunks[0])
    nblks_2d = len(A.chunks[0]), len(A.chunks[1])
  
    # rechunk x0?

    _r = 'r-' + itertoken
    _x = 'x-' + itertoken
    _Ax = 'Ax-' + itertoken
    _p = 'p-' + itertoken
    _Mr = 'Mr-' + itertoken
    _resnrm2 = 'resnrm2-' + itertoken
    
    if x0 is None:
        dsk_r = da.core.top(lambda bi: bi, _r, 'i', b.name, 'i', 
                            numblocks={b.name: nblks_1d})
        dsk_x = da.core.top(lambda ri: 0 * ri, _x, 'i', _r, 'i', 
                            numblocks={_r: nblks_1d})
        dsk_Ax = None
    else:
        dsk_x = da.core.top(lambda x0i: x0i, _x, 'i', x0.name, 'i', 
                            numblocks={x0.name: nblks_1d})
        dsk_Ax = da.core.top(da.core.dotmany, _Ax, 'i', A.name, 'ij', _x, 'j', 
                             numblocks={A.name: nblks_2d, _x: nblks_1d})
        dsk_r = da.core.top(operator.sub, _r, 'i', b.name, 'i', _Ax, 'i',
                            numblocks={b.name: nblks_1d, _Ax: nblks_1d})
        
    if M is None:
        dsk_Mr = {(_Mr, key[1]): dsk_r[_r, key[1]] for key in dsk_r}
    else:
        dsk_Mr = da.core.top(da.core.dot_many( ))
    
    dsk_p = {(_p, key[1]): dsk_Mr[_Mr, key[1]] for key in dsk_Mr}
    dsk_resnrm2 = da.core.top(da.core.dotmany, _resnrm2, '', _r, 'i', _Mr, 'i',
                              numblocks={_r: nblks_1d, _Mr: nblks_1d})
    
    dsk = dask.sharedict.merge(A.dask, b.dask)
    dsk.update_with_key(dsk_x, _x)
    if dsk_Ax is not None:
        dsk.update_with_key(dsk_Ax, _Ax)
    dsk.update_with_key(dsk_r, _r)
    dsk.update_with_key(dsk_p, _p)
    dsk.update_with_key(dsk_resnrm2, _resnrm2)


    x = da.Array(dsk, _x, shape=b.shape, chunks=b.chunks, dtype=b.dtype)



['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_get_nonrecursive',
 '_get_recursive',
 '_toposort',
 'absolute_import',
 'add',
 'chain',
 'division',
 'flatten',
 'get',
 'get_dependencies',
 'get_deps',
 'getcycle',
 'has_tasks',
 'inc',
 'isdag',
 'ishashable',
 'istask',
 'literal',
 'preorder_traversal',
 'print_function',
 'quote',
 'reverse_dict',
 'subs',
 'toposort']

### iteration

In [30]:
def block_cg_iterate(dsk, A, M, resnrm2, iteration, name=None):
    dsk_iter = dict()
    m, n = A.shape
    assert m == n
    chunks_1d = (A.chunks[1],)
    nblks_2d = vblocks, hblocks = len(A.chunks[0]), len(A.chunks[1])
    nblks_1d = (vblocks,)

    token = name or dask.base.tokenize(A)
    itertoken = 'cg-iter-' + str(iteration) + '-' + token
    oitertoken = 'cg-iter-' + str(iteration - 1) + '-' + token

    _Ap = 'Ap-' + itertoken
    _alpha = 'alpha-' + itertoken
    _beta = 'beta-' + itertoken
    _gamma = 'gamma-' + itertoken
    _x = 'x-' + itertoken
    _ox = 'x-' + oitertoken
    _r = 'r-' + itertoken
    _or = 'r-' + oitertoken
    _p = 'p-' + itertoken
    _op = 'p-' + oitertoken
    _Mr = 'Mr-' + itertoken
    _resnrm2 = 'resnrm2-' + itertoken
    _oresnrm2 = 'resnrm2-' + oitertoken

    oresnrm2 = resnrm2

    # alpha = resnrm2 / p.dot(Ap)
    dsk_Ap = da.core.top(da.core.dotmany, _Ap, 'i', A.name, 'ij', _op, 'j',
                         numblocks={A.name: nblks_2d, _op: nblks_1d})
    dsk_gamma = da.core.top(da.core.dotmany, _gamma, '', _op, 'i', _Ap, 'i', 
                            numblocks={_op: nblks_1d, _Ap: nblks_1d})
    dsk_alpha = da.core.top(np.divide, _alpha, '', _oresnrm2, '', _gamma, '',
                            numblocks={_oresnrm2: (), _gamma: ()})

    # x = ox + alpha * p
    def update_x(xi, pi, alpha): return xi + alpha * pi
    dsk_x = da.core.top(update_x, _x, 'i', _ox, 'i', _op, 'i', _alpha, '',
                        numblocks={_ox: nblks_1d, _op: nblks_1d, _alpha: ()})

    # r = ores - alpha * Ap
    def update_r(ri, Api, alpha): return ri - alpha * Api
    dsk_r = da.core.top(update_r, _r, 'i', _or, 'i', _op, 'i', _alpha, '',
                        numblocks={_or: nblks_1d, _op: nblks_1d, _alpha: ()})

    # resnrm2 = r'Mr
    if M is None:
        dsk_Mr = {(_Mr, rkey[1]): dsk_r[_r, rkey[1]] for rkey in dsk_r}
    else:
        dsk_Mr = da.core.top(da.core.dot_many, _Mr, 'i', M.name, 'ij', _r, 'j',
                             numblocks={M.name: nblks_2d, _r: nblks_1d})
    dsk_resnrm2 = da.core.top(da.core.dotmany, _resnrm2, '', _r, 'i', _Mr, 'i',
                              numblocks={_r: nblks_1d, _Mr: nblks_1d})

    # p = Mr + (resnrm2 / oresnrm2) * op
    dsk_beta = da.core.top(np.divide, _beta, '', _resnrm2, '', _oresnrm2, '',
                           numblocks={ _resnrm2: (), _oresnrm2: ()})
    def update_p(Mri, pi, beta): return Mri + beta * pi
    dsk_p = da.core.top(update_p, _p, 'i', _Mr, 'i', _op, 'i', _beta, '',
                          numblocks={_Mr: nblks_1d, _op: nblks_1d, _beta: ()})

    dsk = dask.sharedict.merge(dsk, (itertoken, dsk_iter))
    dsk.update_with_key(dsk_Ap, key=_Ap)
    dsk.update_with_key(dsk_gamma, key=_gamma)
    dsk.update_with_key(dsk_alpha, key=_alpha)
    dsk.update_with_key(dsk_x, key=_x)
    dsk.update_with_key(dsk_r, key=_r)    
    dsk.update_with_key(dsk_Mr, key=_Mr)
    dsk.update_with_key(dsk_resnrm2, key=_resnrm2)
    dsk.update_with_key(dsk_beta, key=_beta)
    dsk.update_with_key(dsk_p, key=_p)

    x = da.Array(dsk, _x, shape=(m,), chunks=chunks_1d, dtype=A.dtype)
    r = da.Array(dsk, _r, shape=(m,), chunks=chunks_1d, dtype=A.dtype)
    p = da.Array(dsk, _p, shape=(m,), chunks=chunks_1d, dtype=A.dtype)
    resnrm2 = da.Array(dsk, _resnrm2, shape=(), chunks=(), dtype=A.dtype)

    x, r, p, resnrm2 = dask.persist(x, r, p, resnrm2)
    dsk = dask.sharedict.merge(x.dask, r.dask, p.dask, resnrm2.dask)
    
    return dsk, x, resnrm2

In [31]:
dsk, x, resnrm2 = block_cg_initialize(A, b, M, x0, name=name)

In [32]:
dsk, x, resnrm2 = block_cg_iterate(dsk, A, M, resnrm2, 1, name=name)

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [52]:
dask.compute(resnrm2)

(0.000950357395348805,)

In [53]:
import time
dsk, x, resnrm2 = block_cg_initialize(A, b, M, x0, name=name)
for i in range(1, 501):
    start = time.time()
    dsk, x, resnrm2 = block_cg_iterate(dsk, A, M, resnrm2, i, name=name)
    (res,) = dask.compute(resnrm2)
    print i, res**0.5
    if resnrm2**0.5 < 1e-5:
        break
    print time.time() - start

TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''